In [1]:
import torch
print(torch.cuda.is_available())  
print(torch.cuda.current_device())  
print(torch.cuda.get_device_name(torch.cuda.current_device())) 

True
0
NVIDIA GeForce RTX 4060 Ti


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import numpy as np


In [4]:
df_train = pd.DataFrame(np.random.rand(700, 42), columns=[f"feature_{i}" for i in range(42)])
df_test = pd.DataFrame(np.random.rand(300, 42), columns=[f"feature_{i}" for i in range(42)])

y_train = (np.random.rand(700) > 0.5).astype(np.float32)
y_test = (np.random.rand(300) > 0.5).astype(np.float32)


scaler = MinMaxScaler()
X_train = scaler.fit_transform(df_train.values)
X_test = scaler.transform(df_test.values)

# y_train = y_train.values.astype(np.float32)
# y_test = y_test.values.astype(np.float32)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


torch.Size([700, 42]) torch.Size([700]) torch.Size([300, 42]) torch.Size([300])


In [5]:
y_train

tensor([1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
        1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
        0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
        0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1.,
        1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1.,
        0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
        1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
        0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 

In [6]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [7]:
tabnet_params = {
    "n_d": 16,
    "n_a": 16,
    "n_steps": 3,
    "n_shared": 2,
    "n_independent": 2,
    "gamma": 1.3,
    "epsilon": 1e-15,
    "momentum": 0.98,
    "mask_type": "sparsemax",
    "lambda_sparse": 1e-3,
    "device_name": "cuda" if torch.cuda.is_available() else "cpu"
}


unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    **tabnet_params
)
 

unsupervised_model.fit(
    X_train,
    eval_set=[X_test],  
    pretraining_ratio=0.8,
    max_epochs=101,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

c:\ProgramData\miniconda3\envs\pytorch_gpu\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 61.83712| val_0_unsup_loss_numpy: 10.790340423583984|  0:00:00s
epoch 1  | loss: 42.11308| val_0_unsup_loss_numpy: 8.697540283203125|  0:00:00s
epoch 2  | loss: 31.71457| val_0_unsup_loss_numpy: 6.553380012512207|  0:00:00s
epoch 3  | loss: 23.83385| val_0_unsup_loss_numpy: 5.704319953918457|  0:00:00s
epoch 4  | loss: 18.34557| val_0_unsup_loss_numpy: 4.987850189208984|  0:00:00s
epoch 5  | loss: 15.03526| val_0_unsup_loss_numpy: 4.498489856719971|  0:00:00s
epoch 6  | loss: 11.29252| val_0_unsup_loss_numpy: 3.9113500118255615|  0:00:00s
epoch 7  | loss: 9.24485 | val_0_unsup_loss_numpy: 3.2358601093292236|  0:00:00s
epoch 8  | loss: 7.54443 | val_0_unsup_loss_numpy: 3.0275800228118896|  0:00:00s
epoch 9  | loss: 6.0153  | val_0_unsup_loss_numpy: 2.6784799098968506|  0:00:00s
epoch 10 | loss: 4.90302 | val_0_unsup_loss_numpy: 2.3690900802612305|  0:00:00s
epoch 11 | loss: 3.99891 | val_0_unsup_loss_numpy: 1.9045599699020386|  0:00:00s
epoch 12 | loss: 3.15261 | val_0_

c:\ProgramData\miniconda3\envs\pytorch_gpu\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [8]:
# Truy cập vào mô hình TabNet bên trong
from torchinfo import summary

tabnet_model = unsupervised_model.network.to(device)

summary(tabnet_model, input_size=X_train.shape) 

Layer (type:depth-idx)                                       Output Shape              Param #
TabNetPretraining                                            [700, 42]                 --
├─EmbeddingGenerator: 1-1                                    [700, 42]                 --
├─TabNetEncoder: 1-2                                         [700, 16]                 --
│    └─BatchNorm1d: 2-1                                      [700, 42]                 84
│    └─FeatTransformer: 2-2                                  [700, 32]                 4,352
│    │    └─GLU_Block: 3-1                                   [700, 32]                 4,992
│    └─ModuleList: 2-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-17                            --                        (recursive)
│    └─FeatTransformer: 2-6                                  --                        (recursive)
│    │    └─GLU_Block: 3-5                                   -

In [9]:
encoder = tabnet_model.encoder

print("\nEncoder Summary:")
print(encoder)




Encoder Summary:
TabNetEncoder(
  (initial_bn): BatchNorm1d(42, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  (initial_splitter): FeatTransformer(
    (shared): GLU_Block(
      (shared_layers): ModuleList(
        (0): Linear(in_features=42, out_features=64, bias=False)
        (1): Linear(in_features=32, out_features=64, bias=False)
      )
      (glu_layers): ModuleList(
        (0): GLU_Layer(
          (fc): Linear(in_features=42, out_features=64, bias=False)
          (bn): GBN(
            (bn): BatchNorm1d(64, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
          )
        )
        (1): GLU_Layer(
          (fc): Linear(in_features=32, out_features=64, bias=False)
          (bn): GBN(
            (bn): BatchNorm1d(64, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
          )
        )
      )
    )
    (specifics): GLU_Block(
      (glu_layers): ModuleList(
        (0-1): 2 x GLU_Layer(
          (fc): Linear(in_fea

In [10]:
decoder = tabnet_model.decoder

print("\nDecoder Summary:")
print(decoder)


Decoder Summary:
TabNetDecoder(
  (feat_transformers): ModuleList(
    (0-2): 3 x FeatTransformer(
      (shared): GLU_Block(
        (shared_layers): ModuleList(
          (0): Linear(in_features=16, out_features=32, bias=False)
        )
        (glu_layers): ModuleList(
          (0): GLU_Layer(
            (fc): Linear(in_features=16, out_features=32, bias=False)
            (bn): GBN(
              (bn): BatchNorm1d(32, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
            )
          )
        )
      )
      (specifics): GLU_Block(
        (glu_layers): ModuleList(
          (0): GLU_Layer(
            (fc): Linear(in_features=16, out_features=32, bias=False)
            (bn): GBN(
              (bn): BatchNorm1d(32, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
            )
          )
        )
      )
    )
  )
  (reconstruction_layer): Linear(in_features=16, out_features=42, bias=False)
)


In [11]:
sample_input = torch.tensor(X_train[:5]).to(device)  

try:
    result = tabnet_model.encoder(sample_input)
    if isinstance(result, tuple):
        print(f'TabNet encoder trả về {len(result)} giá trị.')
        for i, res in enumerate(result):
            print(f'Giá trị {i + 1} shape: {res.shape}')
    else:
        print('TabNet encoder chỉ trả về một giá trị.')
        print(f'Giá trị shape: {result.shape}')
except Exception as e:
    print(f'Đã xảy ra lỗi: {e}')

TabNet encoder trả về 2 giá trị.
Đã xảy ra lỗi: 'list' object has no attribute 'shape'


C:\Users\PC\AppData\Local\Temp\ipykernel_2664\3356785120.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample_input = torch.tensor(X_train[:5]).to(device)


In [12]:
class Sampling(nn.Module):
    def __init__(self, seed=1337):
        super(Sampling, self).__init__()
        self.seed = seed

    def forward(self, inputs):
        z_mean, z_log_var = inputs
        batch = z_mean.size(0)
        dim = z_mean.size(1)
        # print(batch, dim)
        epsilon = torch.randn(batch, dim, generator=torch.Generator().manual_seed(self.seed)).to(device)
        return z_mean + torch.exp(0.5 * z_log_var) * epsilon

In [13]:
class VAE_Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(VAE_Encoder, self).__init__()
        self.tabnet_encoder = tabnet_model.encoder
        self.mlp = nn.Sequential(
            nn.Linear(16, 128),  
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 96),
            nn.ReLU(),
            nn.Linear(96, latent_dim)
        ).to(device)
        self.fc_mean = nn.Linear(latent_dim, latent_dim).to(device)
        self.fc_log_var = nn.Linear(latent_dim, latent_dim).to(device)
        self.sampling = Sampling().to(device)

    def forward(self, x):
        x = x.to(device)
        steps_output, _ = self.tabnet_encoder(x)
        encoded = steps_output[-1]
        # print("Shape of encoded tensor:", encoded.shape)
        encoded = self.mlp(encoded)
        z_mean = self.fc_mean(encoded)
        z_log_var = self.fc_log_var(encoded)
        z = self.sampling((z_mean, z_log_var))
        # print(f'Shape of z: {z.shape} - {z_log_var.shape} -{z_log_var.shape}')
        return z_mean, z_log_var, z


In [14]:
class VAE_Decoder(nn.Module):
    def __init__(self, latent_dim,encoded_dim, output_dim):
        super(VAE_Decoder, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(latent_dim, 32),   
            nn.ReLU(),
            nn.Linear(32, 96),
            nn.ReLU(),
            nn.Linear(96, 96),
            nn.ReLU(),
            nn.Linear(96, encoded_dim),  
        )
        self.tabnet_decoder = tabnet_model.decoder
        self.reshape = nn.Unflatten(1, (encoded_dim,))
        self.output_dim=output_dim


    def forward(self, z):
        x = F.relu(self.mlp(z))

        # print("Shape before reshape:", x.shape)
        # x = self.reshape(x)
        x = x[None, ...]

        # print("Shape after reshape:", x.shape)
        # x = x.view(x.size(0), output_dim)
        
        output = self.tabnet_decoder(x)
        # print(output.shape)
        # print("Shape of output from tabnet_decoder:", output.shape)
        output = torch.sigmoid(output)
        output = output.view(-1, self.output_dim)
        return output

In [15]:
def check_data_range(tensor, name):
    if not torch.all((tensor >= 0) & (tensor <= 1)):
        print(f"{name} contains values outside the range [0, 1]")
        print(f"{name} min: {tensor.min()}, max: {tensor.max()}")

In [16]:
class VAE_Tabnet_MLPS(nn.Module):
    def __init__(self, encoder, decoder, classifier):
        super(VAE_Tabnet_MLPS, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.classifier = classifier
        self.total_loss_tracker = []
        self.reconstruction_loss_tracker = []
        self.kl_loss_tracker = []
        self.classification_loss_tracker = []
        self.accuracy_tracker = []

    def forward(self, x):
        z_mean, z_log_var, z = self.encoder(x)
        reconstruction = self.decoder(z)
        classification_output = self.classifier(z)
        return reconstruction, z_mean, z_log_var, classification_output

    def train_step(self, data, labels, optimizer):
        optimizer.zero_grad()
        # z_mean, z_log_var, z = self.encoder(data)
        # reconstruction = self.decoder(z)
        reconstruction, z_mean, z_log_var, classification_output = self.forward(data)
        # print('classifi',classification_output.shape)
        # print(check_data_range(data, 'data'))
        # print(check_data_range(reconstruction, 'reconstruction'))
        # reconstruction_loss = torch.mean(
        #     torch.sum(
        #         F.binary_cross_entropy(reconstruction, data, reduction='none'),
        #         dim=1
        #     )
        # )
        reconstruction_loss = torch.mean(
            torch.sum(
                F.binary_cross_entropy_with_logits(reconstruction, data, reduction='none'),
                dim=1
                # dim=(1, 2)
                )  
        )
        classification_loss = torch.mean(
            torch.sum(
                F.binary_cross_entropy_with_logits(classification_output, labels, reduction='none'),
                # dim=1
                # dim=(1, 2)
                )  
        )
        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp(), dim=1)
        kl_loss = torch.mean(torch.sum(kl_loss))
        total_loss = reconstruction_loss + kl_loss + classification_loss
        total_loss.backward()
        optimizer.step()

        self.total_loss_tracker.append(total_loss.item())
        self.reconstruction_loss_tracker.append(reconstruction_loss.item())
        self.kl_loss_tracker.append(kl_loss.item())
        self.classification_loss_tracker.append(classification_loss.item())

        preds = torch.sigmoid(classification_output)
        correct = ((preds > 0.5) == labels).float().sum()
        accuracy = correct / labels.size(0)
        self.accuracy_tracker.append(accuracy.item())

        return {
            "loss": total_loss.item(),
            "reconstruction_loss": reconstruction_loss.item(),
            "kl_loss": kl_loss.item(),
            "classification_loss": classification_loss.item(),
            "accuracy": accuracy.item()
        }

In [17]:
latent_dim = 64
encoded_dim = 16
output_dim = X_train.shape[1]
input_dim = X_train.shape[1]
print(input_dim)


42


In [18]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, output_dim),
            nn.Sigmoid()  
        )

    def forward(self, x):
        # print('input: ',x.shape)
        output = self.fc(x)
        output = output.view(-1)
        # print('output',output.shape)
        return output

In [19]:
classifier = SimpleClassifier(latent_dim, output_dim=1).to(device)


In [20]:
def check_output(model, input_tensor):
    with torch.no_grad():  
        output = model(input_tensor)
        print(f"Input size: {input_tensor.size()}")
        print(f"Output size: {output.size()}")
        print(f"Output: {output}")

model = SimpleClassifier(latent_dim, output_dim=1)

input_tensor = torch.randn(32,latent_dim)  

check_output(model, input_tensor)

Input size: torch.Size([32, 64])
Output size: torch.Size([32])
Output: tensor([0.4993, 0.5123, 0.5001, 0.4784, 0.4995, 0.5212, 0.5218, 0.5042, 0.5236,
        0.5099, 0.5078, 0.4799, 0.5123, 0.5068, 0.5320, 0.5113, 0.5377, 0.4922,
        0.5366, 0.5090, 0.5213, 0.5124, 0.4964, 0.5286, 0.5102, 0.4829, 0.5065,
        0.4959, 0.5000, 0.5307, 0.4999, 0.5316])


In [22]:
vae_encoder = VAE_Encoder(latent_dim=latent_dim)
print("Encoder Summary:")
# vae_encoder.to(device)

summary(vae_encoder, input_size=(32, input_dim), device=device)

Encoder Summary:


Layer (type:depth-idx)                                       Output Shape              Param #
VAE_Encoder                                                  [32, 64]                  --
├─TabNetEncoder: 1-1                                         [32, 16]                  --
│    └─BatchNorm1d: 2-1                                      [32, 42]                  84
│    └─FeatTransformer: 2-2                                  [32, 32]                  4,352
│    │    └─GLU_Block: 3-1                                   [32, 32]                  4,992
│    └─ModuleList: 2-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-17                            --                        (recursive)
│    └─FeatTransformer: 2-6                                  --                        (recursive)
│    │    └─GLU_Block: 3-5                                   --                        (recursive)
│    └─ModuleList: 2-12                              

In [23]:
x = torch.randn(800, 42).to(device)
steps_output, _ = tabnet_model.encoder(x)
encoded = steps_output[-1]
print(f"Encoded shape: {encoded.shape}")

Encoded shape: torch.Size([800, 16])


In [24]:
import torch

x = torch.randn(800, 42).to(device)  # Đầu vào có kích thước (batch_size, features)

steps_output, _ = tabnet_model.encoder(x)
print("Shape of encoder output:", [output.shape for output in steps_output])

decoder_input = steps_output[-1]  
decoder_input = decoder_input[None, ...]
try:
    decoder_output = tabnet_model.decoder(decoder_input)
    print(f"Decoder shape: {decoder_output.shape}")
except ValueError as e:
    print(f"Error: {e}")


Shape of encoder output: [torch.Size([800, 16]), torch.Size([800, 16]), torch.Size([800, 16])]
Decoder shape: torch.Size([800, 42])


In [26]:
vae_decoder = VAE_Decoder(latent_dim=latent_dim, encoded_dim=encoded_dim, output_dim=output_dim).to(device)
print("Decoder Summary:")
summary(vae_decoder, input_size=(32, latent_dim), device=device)

Decoder Summary:


Layer (type:depth-idx)                                       Output Shape              Param #
VAE_Decoder                                                  [32, 42]                  --
├─Sequential: 1-1                                            [32, 16]                  --
│    └─Linear: 2-1                                           [32, 32]                  2,080
│    └─ReLU: 2-2                                             [32, 32]                  --
│    └─Linear: 2-3                                           [32, 96]                  3,168
│    └─ReLU: 2-4                                             [32, 96]                  --
│    └─Linear: 2-5                                           [32, 96]                  9,312
│    └─ReLU: 2-6                                             [32, 96]                  --
│    └─Linear: 2-7                                           [32, 16]                  1,552
├─TabNetDecoder: 1-2                                         [32, 42]              

In [27]:
vae = VAE_Tabnet_MLPS(encoder=vae_encoder, decoder=vae_decoder,classifier=classifier).to(device)
summary(vae, input_size=(32, input_dim), device=device)

Layer (type:depth-idx)                                            Output Shape              Param #
VAE_Tabnet_MLPS                                                   [32, 42]                  --
├─VAE_Encoder: 1-1                                                [32, 64]                  --
│    └─TabNetEncoder: 2-1                                         [32, 16]                  --
│    │    └─BatchNorm1d: 3-1                                      [32, 42]                  84
│    │    └─FeatTransformer: 3-2                                  [32, 32]                  9,344
│    │    └─ModuleList: 3-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-6                                  --                        (recursive)
│    │    └─ModuleList: 3-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-6                                  --                        (recursive)
│    │

In [28]:
learning_rate = 0.0001
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
num_epochs = 10

for epoch in range(num_epochs):
    vae.train()
    train_loss = 0
    rec_loss = 0
    kl_loss = 0
    classification_loss = 0
    accuracy = 0

    for batch_data, batch_labels in train_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        results = vae.train_step(batch_data, batch_labels, optimizer)
        
        train_loss += results["loss"]
        rec_loss += results["reconstruction_loss"]
        kl_loss += results["kl_loss"]
        classification_loss += results["classification_loss"]
        accuracy += results["accuracy"]

    train_loss /= len(train_loader)
    rec_loss /= len(train_loader)
    kl_loss /= len(train_loader)
    classification_loss /= len(train_loader)
    accuracy /= len(train_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Reconstruction Loss: {rec_loss:.4f}, KL Loss: {kl_loss:.4f}, Classification Loss: {classification_loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/10, Loss: 61.5197, Reconstruction Loss: 30.5955, KL Loss: 7.7059, Classification Loss: 23.2182, Accuracy: 0.4986
Epoch 2/10, Loss: 58.9271, Reconstruction Loss: 30.5393, KL Loss: 5.2059, Classification Loss: 23.1819, Accuracy: 0.4980
Epoch 3/10, Loss: 56.9059, Reconstruction Loss: 30.4935, KL Loss: 3.2842, Classification Loss: 23.1282, Accuracy: 0.4990
Epoch 4/10, Loss: 55.3657, Reconstruction Loss: 30.4687, KL Loss: 1.7847, Classification Loss: 23.1123, Accuracy: 0.4990
Epoch 5/10, Loss: 54.3504, Reconstruction Loss: 30.4578, KL Loss: 0.8443, Classification Loss: 23.0484, Accuracy: 0.4988
Epoch 6/10, Loss: 53.8529, Reconstruction Loss: 30.4381, KL Loss: 0.3711, Classification Loss: 23.0437, Accuracy: 0.4982
Epoch 7/10, Loss: 53.5797, Reconstruction Loss: 30.4264, KL Loss: 0.1611, Classification Loss: 22.9922, Accuracy: 0.4982
Epoch 8/10, Loss: 53.4148, Reconstruction Loss: 30.4118, KL Loss: 0.0847, Classification Loss: 22.9184, Accuracy: 0.4992
Epoch 9/10, Loss: 53.3234, Recon